<a href="https://colab.research.google.com/github/avatar220928/Colab-Github/blob/material/%E5%B0%8F%E5%B7%A5%E5%BB%A0%E4%BA%8C%E8%99%9F_FB_%E5%9C%8B%E5%AE%B6%E5%B1%A4%E7%B4%9A%E7%9B%B8%E9%97%9C%E4%BF%82%E6%95%B8%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **引入需要的套件**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',20)
pd.set_option('max_colwidth',1000)
import numpy as np
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from google.colab import files
import seaborn as sns
from scipy.stats.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import datasets
import requests
from bs4 import BeautifulSoup
from google.colab import files


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#讀兩份FB資料
read_file_1 = '/content/drive/MyDrive/mobility移動資料/FB/movement-range-2022-03-19.txt'
read_file_2 = '/content/drive/MyDrive/mobility移動資料/FB/movement-range-data-2020-03-01--2020-12-31.txt'

#讀成dataframe
df_2021=pd.read_table(read_file_1)
df_2020=pd.read_table(read_file_2)

#黏起來
df_1=df_2020.append(df_2021)
df_1.set_index(pd.to_datetime(df_1['ds'],format='%Y-%m-%d'),inplace=True)
# print(df_1.head(-5))


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


                    ds country polygon_source   polygon_id polygon_name  \
ds                                                                        
2020-03-01  2020-03-01     AGO           GADM  AGO.10.10_1      Lubango   
2020-03-02  2020-03-02     AGO           GADM  AGO.10.10_1      Lubango   
2020-03-03  2020-03-03     AGO           GADM  AGO.10.10_1      Lubango   
2020-03-04  2020-03-04     AGO           GADM  AGO.10.10_1      Lubango   
2020-03-05  2020-03-05     AGO           GADM  AGO.10.10_1      Lubango   
...                ...     ...            ...          ...          ...   
2021-11-23  2021-11-23     ZWE           GADM    ZWE.6.6_1       Zvimba   
2021-12-01  2021-12-01     ZWE           GADM    ZWE.6.6_1       Zvimba   
2021-11-17  2021-11-17     ZWE           GADM    ZWE.8.3_1       Hwange   
2021-11-23  2021-11-23     ZWE           GADM    ZWE.8.3_1       Hwange   
2021-04-27  2021-04-27     ZWE           GADM    ZWE.9.1_1   Beitbridge   

            all_day_bing

# **def: 刪除google sheet裡的nan值**

In [ ]:
def deletenan (sheets,a):
  A = sheets.col_values(a)
  #print(A)
  country_google_name = A.copy() #A是google_name的複製版
  for i in range(len(A)):
    if A[i]== '':
      country_google_name.remove(A[i])
  # print(country_google_name)
  return country_google_name

# **def: 篩選case資料中所需的國家**

In [ ]:
def choose_case(a,case_data,country_test):
  case_data_new=case_data[case_data[ 'location' ]==country_test[a]] #篩選case國家
  return(case_data_new)
  print('成功2!')

# **def: 篩選FB資料中所需的國家**

In [ ]:
def choose_FB(a,FB_data,country_name):
  
  sub_region_set=FB_data[FB_data['country']==country_name[a]] #篩選FB國家
  sub_region_set=set(sub_region_set['polygon_name'])
  sub_region_list=list(sub_region_set)
  sub_region_list_1 = [str(x) for x in sub_region_list]
  sub_region_list_2 = [x for x in sub_region_list_1 if x != 'nan']
  # print('成功!')
  # print(sub_region_list_2)
  return(sub_region_list_2)
  

# **def: 轉換timestamp變成字串**

In [ ]:
def timestamp2string(a):
  #a = a.to_pydatetime()
  str1 = a.strftime('%Y-%m-%d')
  return str1

# **def: 切開日期list、指定日期**

In [ ]:
def split_list(date, n, n_2):
  a = 0
  if n[0]!= 1:
    for  i_6 in range(len(n_2)):
      a = a + n[i_6]
      yield date[a : a + n_2[i_6]]
  else: 
      yield date[0 : 1]

# **def: 計算FB movement和new case之間的相關係數**

In [ ]:
all_case= pd.read_csv('/content/drive/MyDrive/mobility移動資料/all_cases/owid-covid-data.csv')

def googlecorrtable(country_name, country_test):
  for i in range(len(country_name)): #選fb國家(df_1是fb資料)
    print('現在進入到這個國家: '+country_name[i])
    if country_name[i]=='MAR':
      pass
    
    else:
      n = len(start_date[i])
      A = pd.DataFrame(np.zeros((n,4)),columns=['n','place','Coefficient_of_Correlation','p-value'])
      
      
      sub_region_list_2 = choose_FB(i,df_1,country_name) #FB資料篩選
      print(sub_region_list_2)
      case_data_new = choose_case(i, all_case, country_test) #case篩選
      case_data_new.set_index(pd.to_datetime(case_data_new['date'],format='%Y-%m-%d'),inplace=True)

      for ii in range(3): #選fb國家的polygon取前3
        
        df_2 = df_1[df_1['polygon_name']==sub_region_list_2[ii]]
        # temp = [] 
        # for i_2 in df_2:
        #   if df_2['ds'][i_2] not in set(df_2['ds']):
        #     value = df_2['all_day_bing_tiles_visited_relative_change'][i_2]
        #     temp.append(value)
        temp = df_2.copy()
        temp.set_index(pd.to_datetime(temp['ds'],format='%Y-%m-%d'),inplace=True)
        corr=[]
        p=[]
        FB_mobility_sum = pd.DataFrame()
        for iii in range(len(start_date[i])):  #這個迴圈負責把dataframe分解、重排

          temp__= temp[temp["ds"].between(start_date[i][iii],end_date[i][iii])]
          FB_mobility_sum = pd.concat([FB_mobility_sum, temp__['all_day_bing_tiles_visited_relative_change']],axis=1)
          print(FB_mobility_sum)
          print('--------------------------------------------')
      a=[]
      for b in range(0,len(FB_mobility_sum.columns),len(start_date[i])):
        a.append(b)
      aa=a.copy()
      while (len(a)<len(FB_mobility_sum.columns)):
        aa = [x+1 for x in aa]
        a+=aa
        print(a)

      #FB_mobility_sum_rearrange = FB_mobility_sum.iloc[:,a]
      # print('FB_mobility_sum_rearrange在這邊')
      # print(FB_mobility_sum_rearrange)
      c=0

      for iiii in range(len(start_date[i])):
        A['n'][iiii] = str(iiii+1)
        TW_case = case_data_new[case_data_new["date"].between(start_date[i][iiii],end_date[i][iiii])]
        FB_mobility_sum_new = FB_mobility_sum_rearrange.iloc[:,int(c):int(c)+3]
        # print(FB_mobility_sum_new)
        FB_mobility_sum_mean=FB_mobility_sum_new.mean(skipna=True,axis=1)
        print('取完平均',FB_mobility_sum_mean)
        FB_mean = FB_mobility_sum_mean.dropna()
       
        # FB_mean.index = (pd.to_datetime(temp['ds'],format='%Y-%m-%d'),inplace=True)
        # idx = pd.Index(FB_mean)
        # print(idx)
        # FB_mean = FB_mean.index.drop_duplicates(subset=['ds'])
        # res = []
        # for i_2 in FB_mean:
        #   if i_2 not in FB_mean:
        #     value = 
        #     res.append(i)

        # print(FB_mean['ds'])
        
        aa = FB_mean
        bb = TW_case['new_cases']
        # print(np.isnan(aa))
        # print(np.isnan(bb))
        print(type(aa))

        check_for_nan_1 = aa.isnull().any().any()
        print ('FB_mean有nan嗎?'+ str(check_for_nan_1))

        check_for_nan_2 = bb.isnull().any().any()
        print ('case number有nan嗎?'+ str(check_for_nan_2))

        if str(check_for_nan_1)=='True' or str(check_for_nan_2)=='True': 
          nansite_1 = aa.index[np.where(np.isnan(aa))]  #找fb的nan
          nansite_2 = bb.index[np.where(np.isnan(bb))]  #找case的nan
          print(timestamp2string(nansite_1))
          print(timestamp2string(nansite_2))
          for cc in range(len(nansite_2)):
            aa = aa.drop(pd.to_datetime(timestamp2string(nansite_2)[cc]))
          for dd in range(len(nansite_1)):
            bb = bb.drop(pd.to_datetime(timestamp2string(nansite_1)[dd]))

          bb = bb[bb.notna()]
          aa = aa[aa.notna()]

        print(len(aa))
        print(len(bb))
        print(aa)
        print(bb)

        if (len(aa)==len(bb)) and (len(aa)!=0):
          ans = pearsonr(aa, bb)
          corr.append(ans[0])
          p.append(ans[1])
          
        else:
          corr.append(-99)
          p.append(-99)
        print(corr)

        # print(FB_mean)
        # print(TW_case['new_cases'])
        # print(len(TW_case['new_cases']))
        # print(len(FB_mean))
        #print(sub_region_list_2[ii])

        # if len(aa) == len(bb):
        #   ans = pearsonr(aa, bb)
        # # print(ans)
        # corr.append(ans[0])
        # p.append(ans[1])
        

        # FB_mobility_sum.drop(FB_mobility_sum.index,inplace=True)
        c=c+3
      A['Coefficient_of_Correlation'] = corr
      A['p-value'] = p
      A['place']=country_name[i]+'_FB'
      print(A)
      A.to_csv(country_name[i]+'_FB_相關係數.csv',index=False)
        
      


 

In [ ]:
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

urls='https://docs.google.com/spreadsheets/d/1Jxcr8PvzqyEOa3DDT7wh9sBgJDGoDTqqLvDoT1r9R10/edit#gid=337413139'

sheet_name = ['Europe', 'North America','South America','Asia','Africa','Oceania'] 
country_mobility_1 = gc.open_by_url(urls)

for i_1 in range(len(sheet_name)):
  sheet = country_mobility_1.worksheet(sheet_name[i_1])  #讀取國家全名的全部sheet

  #消除nan

  data=[]
  col=[ii for ii in range(1,7)]

  for b in col:
    c = deletenan(sheet,a = b)  #去除每條column的nan值
    data.append(c)

  for bb in range(len(data)):

    int_n = list(map(int, data[3]))  #讓list裡的str變成int
    int_n_2 = int_n.copy()
    
    int_n.insert(0, 0)

  start_date = list(split_list(data[4], int_n, int_n_2))
  end_date = list(split_list(data[5], int_n, int_n_2))

  country_name = data[2] #fb mobility
  country_test = data[0] #case number

  googlecorrtable(country_name, country_test)

現在進入到這個國家: GBR
['England', 'Scotland', 'Northern Ireland', 'Wales']
            all_day_bing_tiles_visited_relative_change
2020-03-02                                     0.03077
2020-03-03                                     0.01637
2020-03-04                                     0.00939
2020-03-05                                    -0.00062
2020-03-06                                     0.02654
...                                                ...
2020-06-28                                    -0.16123
2020-06-29                                    -0.21980
2020-06-30                                    -0.21554
2020-07-01                                    -0.21995
2020-07-02                                    -0.22383

[123 rows x 1 columns]
--------------------------------------------
            all_day_bing_tiles_visited_relative_change  \
2020-03-02                                     0.03077   
2020-03-03                                     0.01637   
2020-03-04                   

InvalidIndexError: ignored